# Aeons documentation

There are 3 different modes:

- Aeons_dummy
    - creates random graph structures with bubbles and tips
    - this was used to generate the initial graphs and other structures but does not use bloom filters yet
- Aeons sim
    - uses longreadsim to generate reads
    - uses bloom filter and generates the graph from that
- Aeons run
    - growing script for in silico runs
    - first place that used read mapping and updating of the graph with new reads


# class structure

## Bloom

a class to hold the bloom filters for an assembly. It actually holds two filter for kmers and k+1-mers that are used
for the score calculation (paths through a node). It also keeps track of all kmers observed so far and holds a set
of all new kmers that can be used to generate new edge lists and edge weights.


## Assembly

takes a bloom object and does an initial construction of a dbg. Can then use a new batch of reads to update that dbg
with new edges

have to make a decision whether I want to go on using graph-tool to maintain an actual graph or just have it implicit
in the bloom filter instead. Would require rebuilding the graph at each batch to calculate the metrics.

Maybe stick with the memory costly approach for now until it gets limiting?

efficiency idea:
instead of saving all observed kmers, save one per read and then recurse into the read?


note about implicit graph from bloom:
- here edge vs node centric makes a big difference. If we save the counts of edges, we reconstruct the nodes.
if we save the counts of nodes, we need to search the neightbors to find the edges, which seems much worse


Problem with graph construction:
when not skipping the creation of edges that are already present,
we end up with two components that are genome sized. But when skipping them, we get N/2 dis-jointed components

For now we will end up with everything duplicated. i.e. each kmer leads to two edges, the additional is from the
reverse complement. But we reduce the size of the observed kmers by only adding new ones
 if their reverse complement is not already in there

IDEA:
if we keep both directions separate, we could have a directed graph which might simplify things.
E.g. the hashimoto would be bigger but much more sparse probably.



## The problem of statis DBGs

Using the data structures we have at the moment, we need to fully reconstruct the dbg whenever we want to update it.
This is not good enough since we need to update the graph in sublinear time whenever we have new reads. And we should
not need to rebuild it every time.

maybe we can still use bloom filters, which can be semi-dynamic by coupling them to a better graph storing structure
or we use a hashed adjacency table instead?

# MPHF and adjacency list with size n * alphabet
maybe with a cuckoo hash to make it possible to extend?

if we use a MPHF, we can still use gt as well. gt uses an adjacency list in C++ afterall,
which should be efficient enough? And if we have integer mappings to kmers,
we should be able to update that in an easier way as well. Then all we need from that adjacency list is a way to
transform it into the hashimoto

nothing that is available seems to be suitable for what I need:
- dynamic minimal perfect hash that can map kmers to indices for an adjacency matrix
- BBHash can be used to generate MPHF, but we need to know all kmers in advance
- CTB wrote a python binding a few months ago that includes a BBHashTable (pybbhash),
    which allows the storage of an index for each kmer
    but again, only useful if you know all kmers in advance
- crawford FDBG is an implementation of Bazzoguy, but only supports limited dynamic operations (still uses an MPHF and
  maps inserted kmers in a much less efficient way. I.e. it expects a very small number of dynamic operations
- bloom filter trie by Holley requires complete traversal, making it a bit useless for lots of dynamic updating


# Idea for dynamic dBG

Build it from multiple components:
- a bloom filter that holds the counts for each kmer
- an adjacency matrix that describes the graph (sparse matrix)
- some hash function that maps the kmers to indices of the matrix
  use khmer and hash compression with simple modulo and don't care too much about collisions for now

- save all observed kmers in a file, and to avoid the need for a reversible hash function,
  at the end we run all kmers through the hash function and check which ones point to the correct indices?


# SparseGraph

building and updating the graph with a sparse matrix works now

also visualisation with gt by looping through entries of matrix and creating edges

can observe the false positive edges from hash collisions very nicely if setting the size of the matrix too small
- maybe implement a different hash func at some point?

or just make the matrix bigger. Not using lil consumes much much less memory, So I can have many more possible indices
to distribute my nodes to.


# updating scores

vectorised the calculation of scores

now works with the visualisation as well. Had to sort the arrays by the order of how the edges were added by gt

# read length distribution

put together a new class (LengthDist) that initialises the prior read length distribution.
keeps track of all observed read lengths with a cheap array of unint16
and can update the distribution and CCL

approximation still needs to be done after we figured out how to calculate U efficiently



# updating the benefit U

to get consecutive scores from each node, we would use the transition matrix up to the lth power
but since the graph is bidirectional we need to use the non-backtracking operator instead
which needs a bit more calculation to arrive at transition probabilities
we calculate both the probability of transitioning to a node and arriving at that node
the first is done with hashimoto turned into a probability matrix
and the second with compl. cml. distribution of read lengths
Another point to consider are dead-ends. Because ccl can not extend further at these positions,
they will have a biased, lowered benefit. Therefore we add some absorbing triangles
to the graph that will propagate ccl at dead-ends

first order of operations: add the absorbers at component ends
absorbers are added and can be visualised as well





